# Airline Partner Recommendation to Electronic Arts 

W261_Spring 2022_FINAL_PROJECT_TEAM 18

Kris Junghee Lee, Cynthia Zhu, Bo He, Bruce Lam

Spring 2022

Section 3


## Sections


- 1 - Question formulation
- 2 - Algorithm Exploration
- 3 - Exploratory data analysis &  Discussion of Challenges
- 4 - Data & Feature Engineering 
- 6 - Algorithm Exploration
- 6 - Algorithm Implementation
- 7 - Conclusion
- 8 - Course Concepts
- 9 - Novelty  
- 10 - Limitaions, challenges, future work

## 1 - Question formulation



Accurate prediction of flight delay is crucial for airlines and their customers because of the cascading consequences of any given flight anomaly on both the carriers and the passengers. Common causes of flight delays include security holdups, threatening weather conditions, maintenance set backs, technical difficulties and flight crew delays. This binary classification of a flight as delayed or not delayed is a prime candidate for decision trees or logistic regression.

Our motivation for creating models to predict flight delays is that we are a data science consultancy hired by Electronic Arts, a large corporation operating in the gaming 
industry to identify and recommend airline carriers to partner with for an exclusivity agreement. Recently, we have seen enormous and long lasting costs being delayed or canceled business flights. As a growing company, EA employees rely on business travel to actively seek opportunities in the market. EA's teams, especially business development, often travel commercially for brand collaborations, partnerships, and aquisitions. However, being late or missing important meetings or events can result in tremendous opportunity cost. Therefore, in addition to providing guidance on airline partnerships, we aim to predict flight delays in advance so that we can help EA avoid missed opportunities company wide by anticipating and making sound decisions when these situations arise. 

The datasets that we will leverage to develop predictive flight delay models include the following:

- [Airline data](https://www.transtats.bts.gov/Fields.asp?gnoyr_VQ=FGJ) from the US Department of Transportation containing flight information like flight number, origin airport, destination airport, time of flight, and delay status between 2015 and 2020.
- Weather data from the National Oceanic and Atmospheric Administration (NOAA) [Integrated Surface Database](https://www.ncei.noaa.gov/data/global-hourly/doc/isd-format-document.pdf) from 2015 to 2020 containing sensor readings on wind speed, visibility, temperature, and dew point.
- A [table of NOAA weather station locations](/mnt/mids-w261/datasets_final_project/stations_data/) that will be the key to joining both the weather and airline datasets

#### Statkeholders & scenario based performance metrics 

Just about everyone travels or is involved with travel for EA business. Stakeholders include everyone from receptionists to security team members who arramge, vet, and schedule or manage meetings to developers and C suite executives. From EA's perspective, we generalized and rank ordered the following travel scenarios, which incorporate a predictive model, from most impactful, worst severity, to least.

- __`Last minute cancellation`__ - Flight and business opportunity missed

|Prediction|Actual|Metric|Consequence|
|----------|-----------|----------------|------------|
|Delayed   |Not delayed|  False Positive| Flight missed if traveler expected delay|
|Not delayed|Delayed   |False Negative|   Late to opportunity, scramble to make changes|
|Delayed   |Delayed    |       True Positive| Late to opportunity, proactive chance to make changes|       
|Not delayed   |Not delayed    |       True Negative| No consequence| 

- In corporate scenario planning, it is very important to manage expectations, false positives can be mitiated through policy requiring employees to arrive at the airport ahead of the originally scheduled, non-delay fight time.
- The next impactful scenario is that of the false negative, which means that the flight was predicted to be on time, but then actualy arrived delayed. In this scenario, travelers have to scramble in hopes to find another option.
- True positives, predicted ahead of time, at least provide a proactive window for the traveler to make new arrangements.

  __`For the majority of EA's business travellers, we optimized modeling around recall because it is the third worst scenatio after last minute cancellation and False Positive, which can be mitigated by policy.`__ 

- Recall: also known as sensitivity, defined as a fraction of correctly identified positives. The metric of choice when minimizing False Negatives.

$$ Recall = \frac{TP}{TP+FN} $$

  __`There are certain travel scenarios that benefit from minimized False Positives, or higher precision, when the consequences are less severe like for conferences, site visits, or team retreats.`__ 

- Precision: proportion of correctly predicted positives of total predicted positives.
$$ Precision = \frac{TP}{TP+FP} $$

For our stakeholders. the cost of misclassifying a non-delay is worse than misclassifying a delay. Therefore, we choose our main evaluator as recall. However, we are going to meet the minimum standard to predict the delay as precisely as possible. Finally, depending on the importance and opportunity lost, we can provide different threshold change the recall/precision weighting. 

#### Continuous usage scenario

In addition to partner recommendation, this team can repurpose the models to predict whether the scheduled flight will be delayed or not up to 6 hrs from 2 hrs before the departure and provide the alarm if the result turns out to be delayed. This will go to their security first, so that they can rearrange meetings. Stakeholders can choose the model to be less conservative or conservative. This means, how strictly to prevent false negatives. 

<img src='https://docs.google.com/uc?export=download&id=1TkleaM3-dVpUfWPbsJeKo-n57arjJYV3' width=50%>

#### State of the art

Surprisingly, despite the prevalence of recommender system across web based services like Amazon and transportation disruptors like Uber and Lime, the airline industry has been slow to adpot [peronalization and dynamic services](https://link.springer.com/article/10.1057/s41272-021-00313-2) for travellers, which would be especially relevant for business travellers who encounter delays. Furthermore, airline performance can be gleaned from social media sentiment, which would be another great input into [delay modeling](http://article.nadiapub.com/IJAST/vol111/10.pdf).

## 2 - Algorithm explanation

### Baseline Model - Decision Tree

Since our objective is to predict flight delays, a binary target of “0” for not delayed and “1” for delayed, based on historical, labeled data, the non-parametric supervised binary classification  approach of decision trees would serve as a good baseline model.

Decision trees approach the challenge of classification by recursively partitioning the data into segments of increasing homogeneity. There are several metrics to measure the homogeneity of a group for this application such as variability/variance for continuous variables, or information gain for classification. In this case, we’ll explore how information gain is used to split, or branch, the data.

The algorithm starts by calculating the total amount of heterogeneity in the target feature, quantified by entropy, which is a measurement of the amount of disorder within that group.

$$Entropy={H} (T)=-\sum _{i=1}^{j}p_{i}\log _{2}p_{i}\$$

Where:

$$p_{i} = \frac{count of class_1}{count of class_1 + count of class_2 }\$$

The next step is to identify the root node, the feature that has the highest information gain, which, in simplified terms, is the ability of that feature to reduce heterogeneity in the data by splitting on a category or threshold value within that group.

<img src='https://miro.medium.com/max/688/1*bcLAJfWN2GpVQNTVOCrrvw.png'/>

source [KD nuggets](https://www.kdnuggets.com/2020/01/decision-tree-algorithm-explained.html)

Information gain is calculated by:

$$\overbrace {IG(T,a)} ^{\text{information gain}}=\overbrace {{H} (T)} ^{\text{entropy (ROOT/decision node)}}-\overbrace {{H} (T\mid a)} ^{\text{sum of entropies (branches)}}\$$

Since we already have the total entropy component, the next step is to compute the information contained within each class within each target feature category, also known as conditional entropy.

$$Information in feature|class of target feature=H(T|a)=\sum \frac{count of class_1 +count of class_2}{total counts of the target feature}*Entropy(a)\$$

This is repeated for every feature in the dataset to determine the feature with the highest information gain, which is set as the root node. An advantage of the tree family of models is that feature selection comes naturally from running the algorithm.

This process continues across each layer of decision nodes until a stopping criterion is achieved. Typical criteria are the maximum depth of the tree (number of decision node layers) or a homogeneity threshold.

The default purity metric for spark decision trees is the Gini index, which takes the place of information gain and also expresses the probability of a feature being classified incorrectly when selected randomly. Expressed as:

$$ Gini Index=1-\sum _{i=1}^{n}(P_i^2) $$

Where \\(P_i\\) is the probability that and observation is classified as one of the target classes.


Tree type algorithms are also highly scalable as they can be implemented without the map phase by including the following 3 optimizations:
- Aggregation of enough child node statistics for each split that don't require parsing many times of the dataset with things like count, sum, sum^2
- Binning of the feature classes by the target classes
- Level-wise training in parallel significantly reduces the of passes through the data, effectively ignoring the data in the preceeding level except for the summary statistics

There are several major trade offs to consider when implementing decision trees 
- A large value for maximum depth can result in overfitting, hindering model generalizability

- Features with a large number of unique values like identification numbers can trick the algorithm as these will likely have high information gain, but no real categorization power
  - The minimum number of samples per split/leaf is also a hyperparameter that can be tuned to help mitigate
- Imbalanced classes can create biases in the tree
- Decision trees alone are generally low bias, high variance models


In order to overcome the trade off of decision tree, we applied gridsearch and compared the result of max depth. Also, we took out unique variable such as ID, date from our dataset. Finally, we applied weight and undersampling to treat the imbanced data set.

### Logistic Regression

Logistic regression is a machine learning binary classification model that leverages the logistic sigmoid activation function.

$$ y =\frac{e^{b_0 +bx}}{1+e^{b_0+bx}} $$

Where y is the dependent variable that is either "1" or "0", b_0 is the intercept term, b is the model coefficient and x is the independent variable.

For the regression model to be trained we need a loss function \\(J(\theta)\\) to be differentiable. And we have used the conventional log loss function as our loss function \\(J(\theta)\\) using below formula. 

$$ J(\theta) = \frac{1}{m} * \sum -y * log(\text{\\^{y}}) - (1-y) * log(1-\text{\\^{y}}) $$

Logistic Regression Gradient Update
The weights of the Logistic regression model are adjusted using the gradient descent steps. At each step, we compute the gradient of the loss function, with respect to the model weights, by using the formula below.

$$Gradient = (\triangle{_W} f(W) = \frac{1}{m} * \sum {x}^{'} * (\text{\\^{y}}-y) $$

Once the gradient is computed for a given step, we adjust the model parameters W, using the equation below:


 $$ W = W - learningRate * (\triangle{_W} f(W)) $$


The learning rate is a hyper-parameter to be tuned, we started with a learning rat e of 0.1 by default, but decided to use 0.2 at the end.

Logistic regression can also leverage L1 and L2 regularization to counter overfitting. Logistic Regression thought to be a good approach, thus, we developed Logistic Regression Model from scratch.

### Ensemble  Model 

In order to improve the model performance from our baseline model and logistic regression model, we explored ensemble methods, which are techniques that combine several base models to produce improved results. There are mainly two types of ensemble model. 


* __Bagging__

Bagging or Bootstrap Aggregation was formally introduced by Leo Breiman in 1996 [3]. Bagging is an Ensemble Learning technique which aims to reduce the error learning through the implementation of a set of homogeneous machine learning algorithms

Example: Random Forest


* __Boosting__

Boosting makes use of a set of base learners to improve the stability and effectiveness of a ML model. The idea behind a boosting architecture is the generation of sequential hypotheses, where each hypothesis tries to improve or correct the mistakes made in the previous one 

Example: Gradient Boost, XGBoost

* __Bagging Vs. Boosting__

After reading related materials, we found out there is not outright winner between Bagging and Boosting models. It depends on the data and circumnstance and our objective.

If the single model gets a lower performance, Bagging will Bagging will rarely get a better bias. However, Boosting could generate a combined model with lower errors as it optimises the advantages and reduces pitfalls of the single model. 

By contrast, if the difficulty of the single model is over-fitting, then Bagging is the best option. Boosting for its part doesn’t help to avoid over-fitting; in fact, this technique is faced with this problem itself. For this reason, Bagging is effective more often than Boosting.

Threfore, we chose the bagging model (Random Forest) and two boosting models(Gradient Boost and XGboost) and examined the difference 

<img src='https://docs.google.com/uc?export=download&id=1TI_ZDDecwrwmaUL82_JOENc3M8EmVpMF' width=50%>

#### Bagging :  Random Forest
To strategically improve the low bias, high variance nature of decision trees, Random Forests reduce the high variance by voting on the outcomes of many randomly generated decision trees,ensembles, thereby cross-validating and pruning resulting in increased bias and generalizability of the model. This enhancement is know as bagging or bootstrap aggregation. The bootstrap phase trains a model by randomly selcting observations with different sets of features with replacement and training a tree. This is done a given number of times by specifiying the bagging parameter. To make predictions, the unseen obsevation with feature set is compared against the average predictions from the regression trees.

$$prediction_(unseen) x'={\frac {1}{B}}\sum _{b=1}^{B}f_{b}(x')$$

Both weather and flights data are highly dimensional, so we leveraged the ability of random forests to rank feature importance using Breiman's methodology. We then selected an importance threshold and eliminated variables below that threshold.

#### Boosting : XGBoost
XGBoost or extreme gradient boosting is a modification on gradient boosting that works on optimizing a Newton-Raphson function instead of a gradient descent function and incorporates a second order Taylor approximation loss function. The methodology claims to improve on regularization, adds extra hyperparameters to enhance randomness of tree creation and claims to be more performant on distributed systems.

#### Boosting : Gradient Boosting Machines (GBM)
To further enhance performance of tree based algorithms, gradient boosting machines (GBM) can be implemented. GBMs act on the "weak learners", models with performance comparable to random guessing. Similar to the random forest methodology of creating ensembles of decision trees, GBMs can target the trees that have low bias and high variance and fit those selected models to "pseudo-redisdual", resulting in stronger learners that can be enhance overall prediction. This often outperforms random forests, and increases the liklihood of overfitting. Regularization is an especially important consideration in GBM with hyperparameters like the number of boosting iterations and adjustment of the learning rate. Like convex gradient descent, choosing a learning rate that is too high might accelerate convergence at the cost of oscillations and potential local minimizations. Conversely, choosing a learning rate that is too slow will result in lengthy optimization times and potentially insufficient intertia to escape from local minima.

### References

Decision trees
- https://www.yumpu.com/en/document/read/37104691/scalable-distributed-decision-trees-in-spark-made-das-sparks-talwalkar
- https://en.wikipedia.org/wiki/Decision_tree_learning
  - Witten, Ian; Frank, Eibe; Hall, Mark (2011). Data Mining. Burlington, MA: Morgan Kaufmann. pp. 102–103. ISBN 978-0-12-374856-0.
- W261 Module 11 on distributed decision trees

Random forest
- https://en.wikipedia.org/wiki/Random_forest
  - Breiman L (2001). "Random Forests". Machine Learning. 45 (1): 5–32. Bibcode:2001MachL..45....5B. doi:10.1023/A:1010933404324.
- https://quantdare.com/what-is-the-difference-between-bagging-and-boosting/
- https://dataaspirant.com/ensemble-methods-bagging-vs-boosting-difference/#t-1599488265677

Gradient boosting
- https://en.wikipedia.org/wiki/Gradient_boosting
  - Cheng Li. "A Gentle Introduction to Gradient Boosting" (PDF).
- https://xgboost.readthedocs.io/en/stable/tutorials/model.html
- https://en.wikipedia.org/wiki/XGBoost
  - Chen, Tianqi; Guestrin, Carlos (2016). "XGBoost: A Scalable Tree Boosting System". In Krishnapuram, Balaji; Shah, Mohak; Smola, Alexander J.; Aggarwal, Charu C.; Shen, Dou; Rastogi, Rajeev (eds.). Proceedings of the 22nd ACM SIGKDD International Conference on Knowledge Discovery and Data Mining, San Francisco, CA, USA, August 13-17, 2016. ACM. pp. 785–794. arXiv:1603.02754. doi:10.1145/2939672.2939785.

Logistic regression
- https://en.wikipedia.org/wiki/Logistic_regression
- http://faculty.cas.usf.edu/mbrannick/regression/Logistic.html
- Gentle, James E., Wolfgang Karl Härdle, and Yuichi Mori, eds. Handbook of computational statistics: concepts and methods. Springer Science & Business Media, 2012.

## 3 - Exploratory data analysis (EDA) & challenges

-   ##### __`Understanding Data Set`__

-   ##### __`Airport Network`__

-   ##### __`Delay Status and Pattern`__
  
-   ##### __`Feature Histogram`__
  
-   ##### __`Gap Analysis by target variable through Box-plot`__
  
-   ##### __`Correlation Matrix`__



<img src='https://docs.google.com/uc?export=download&id=1I8MsqS1Db6IkWX3jguDgNLnAofJOY85Y' width=50%>

#### [Please use this to link to the EDA notebook](https://adb-731998097721284.4.azuredatabricks.net/?o=731998097721284#notebook/4070574710017575)

## 4 - Data & Feature Engineering

-   ##### __`Master Table Join`__
>     
   We have received raw data in approximately 5 million(m) rows by 10 columns of station data, 631m rows by 175 columns of weather station data, to combined with three months of airline flight data (161 thousands rows by 107 columns) and 63.5m rows by 107 columns for full airline flight data set. We have performed following steps to join all of the data together for EDA, Feature selections and modeling.
   
       &check; Deduplication of airline flight data  
       &check; Converting flight depature time from local time into UTC time to match weather data    
       &check; Filter out weather data based on the flight departure dates      
       &check; Split out the condense weather feature columns into granular weather features for EDA  
       &check; Select only the closest weather station data to the departure and ariival airport considering recency       
       &check; Use the weather data snapshot at least 2 hours before the scheduled departure time as our weather features    
       &check; Baseline join using 3m data: ~ 1.45 hours   
       &check; Partition airline data by flight year and station   
       &check; Partition weather data by year and station  
       &check; Master join using by station and year: ~ 50 minutes, without year condition: ~ 10.86 hours
>
>
-   #####__`Feature Selection`__
>     
   After master table was created, we still needed to reduce the size of features. We decided to reduce the size of not relavant features from our dataset through EDA. Additionaly, we ran Random Forest classifier with 3 months dataset first and filtered out many columns which scored zero importance.Final selection of features is as below.     
   
       &check; Seasonlity : Month, Day of Month, Day of Week, Year    
       &check; Airline related : Carrier Name, Carrier ID, Tail Number       
       &check; Origion and Destination: Origion, Destination, Latitude, Longtitude, Origion State, Destination State     
       &check; Weather realted : Visibility distance, Dewpoint temperture, wind speed, wind angle, air temperture, ATMOSPHERIC-PRESSURE-OBSERVATION code, ATMOSPHERIC-PRESSURE-CHANGE code, 
       sea level  atmosphere pressue, ceiling height code, 
       ATMOSPHERIC-PRESSURE-CHANGE code        
       &check; Traffic : Airport Weight      
       &check; Delay Status : delay count, delay count normalization      
       &check; Clock: Elasped time capturing weather, Elasped time caturing delay count     
    - Class Named: *FeatureSelector*
>    
>    
-   ##### __`Derived Variable Generation`__
>     
   After we tested with basic baseline model, we found out the performance was not as good as we expected. Therefore, we generated new variables using pagerank algorithm we learned from the class. Through this, we caculated the weight of airport by looking at the origion and destination traffic. Also, we derived the new features called 'count', 'count_normal'. This refers the delay status of origion airport from 3-6 hrs advance. Also its normalization value. These new variables became good features to predict the delay, and showed the biggest correlation with delay. 
    - Class Named: *PageRanker*,*PageRankAdder*
>    
-   ##### __`Data Cleansing `__
>     
   For modeling, we needed to clean data throughly. First,there are duplicated rows from flight table so got rid of these records.  Also, we had to take out the cancellation flights and those flight having null value in delay or arrival time.  Also some of weather value had 9999, 99999 values and it could affect the Imputer. Therefore, we converted those values into None type. Finally,  some of columns needed to convert into String or  Double type. This was important since String type will go into StringIndexer and Onehot encoded later.   
    - Class Named: *DataCleaner*, *TypeHandler*, *WeightAssigner*
>    
-   ##### __`Null Value Treatment`__
>     
   For String columns, we converted null value into 'Missing' since the One-hot-encoder does not allow us to have null value. For the numerical features, We used Imputer fuction to replace null value into "Median'. The reason we did not want to replace with the mean was to prevent the bias coming from outlier sample. In order to caculate the mean, we divided the dataset into train and test first. Later, we fit Imputer with only train set, and transform the test data. In that way, we could avoid the data leakage issue. 
     - Class Named: *CustomImputer*
-   ##### __`Data Transformation`__
>     
   In order to train ML models like logistic regression and decision trees, we had to use VectorAssembler. Prior to VectorAssember, we needed to transform String type features into One hot encoded using Stringindexer. And then, we applied Vectorassember to vectorize the features.  
    - Class Named: *VectorAssembler*,*StringIndexer*
>    
>   
   


We have made into data engineering part as class except Stringindexer and VectorAssembelr(build-in class) and made them into pipelie. Below is the link of notebook.

#### [Please use this to link to the Data & Feature Engineering notebook](https://adb-731998097721284.4.azuredatabricks.net/?o=731998097721284#notebook/1858507102429804/command/1858507102429805)

## 5 - Algorithm Exploration and Model Enhancement

#### Model Enhancment Measure

While we were working on inital model, and explopring the data, we found out that the class is very imbalanced as below table. 
This imbalaned set caused the lower performance of model prediction. In order to tackle this, we implenmened below approach.

| Type         | Sample Count | 
|-------------| --------|
| Non-delayed         | 18681952 | 
| Delayed         | 4156312 | 

-   ##### __`Imbalanced DataSet`__
>     
    - __Weight__ : We gave weight to minority class to overcome imbalanced ratio. 
    - __Undersampling__ : The other way we tried is undersampling. We undersampled the marjoity class sample
   
>   
-   ##### __`Timeseries Data Bias Handling`__
>     
    - __Time-Series Custom Cross Validation__ :  K-fold cross validation splits the training set into k smaller sets and a model is trained using k-1 folds of the training data. However, this apporach has some servere problem in Time-series data in terms of Data leakage. Therefore, we devloped or custom cross valdiation code to apply into our data set. Between *SlidingWindowSplitter* Versus *ExpandingWindowSplitter* , we chose SlidingWindowSplitter considering latte one is computationally expensive and slower
    
     
>  

-   ##### __`Performance Tuning`__
>     
    - __GridSearch__ :    We developed the custom parameter tuning code to test the difference performance by paraemter. For the tree model, we usually looked at the depth of tree, and minimum samples. The final performance is the output of time-sereis cross vadliation
    - __Parameter Moving__ :      After we applied the best parameter from Gridsearch, we adjusted the threshold to get the best balanced output. Precision and Recall is trade off relationship. Therefore, we have to carefully choose ideal point that can meet the minimum criteria of precision at the same time better recall. Below is the example of XGboost. If we put higher threshold than 0.5, you will see the recall gets decresed dramatically. Thefore, we ajudsted threshold as 0.4 
   
<img src='https://docs.google.com/uc?export=download&id=1vw_MJG-ihNjQsJgnMPSftvyhzSLIjnpE' width=30%>


*-SlidingWindowSplitter* : This splitter generates folds across a sliding window over time.

*-ExpandingWindowSplitter* : the ExpandingWindowSplitter generates folds across a sliding window over time. But, the length of the training series grows over time

Reference: https://towardsdatascience.com/dont-use-k-fold-validation-for-time-series-forecasting-30b724aaea64

### Model Optimazation 

Starting from Initial model, and applying multiple methods, below is the final summary of Model Optimazation and Best model

#### Decision tree (Baseline Model)
__Evaluation results in each tuning step__:

| Model         | undersampling | precision |&nbsp;&nbsp; recall &nbsp;&nbsp;   | f1-score | params | Threshold |
| ------------- | ---------| --------- | ---------                            | --------- | ------- |-------        |
| Initial       | No | 0.55      | 0.08                                       | 0.14      | maxDepth=5  | 0.5 |
| Grid Search   | No | 0.54      | 0.08                                      | 0.13      | maxDepth = 3, minWeightFractionPerNode=0, minInstancesPerNode=1 | 0.5



#### XGboost 
__Evaluation results in each tuning step__:


| Model               | undersampling | class weight | precision |&nbsp;&nbsp; recall &nbsp;&nbsp;   | f1-score | params | Threshold
| ------------------- | --------------|--------------| --------- | ---------                        | --------- |--------- |--------- 
| Initial             | No            |  No         | 0.61      | 0.09                              | 0.16      |   maxDepth = 3     | 0.5
| Grid Search         |No             |  Yes         | 0.22      | 0.88                             | 0.36      |  maxDepth = 2, scale_pos_weight = 8, min_child_weight = 2   | 0.5
| Final Candidate     | No            |  Yes         | 0.24      | 0.8                              | 0.37      |  maxDepth = 2, scale_pos_weight = 8, min_child_weight = 2   | 0.55
| Initial             | Yes           |  No         | 0.29      | 0.62                              | 0.40      | maxDepth = 2, scale_pos_weight = 8, min_child_weight = 2   | 0.5
| Final Candidate | Yes           |  No         |  0.27     | 0.72                              |   0.39    | maxDepth = 2, scale_pos_weight = 8, min_child_weight = 2   | 0.4

#### Gradient Boost
__Evaluation results in each tuning step__:

| Model              | undersampling | class weight | precision |&nbsp;&nbsp; recall &nbsp;&nbsp;   | f1-score | Params    | Threshold 
| ------------------ | --------------| ------------ | --------- | --------------------------------- | -------- |--------   | --------
| Initial #1         | No            | No           | 0.58         | 0.08                                 | 0.15        | maxDepth=5  | 0.5
| Initial #2        | No            | Yes           | 0.28         | 0.6                                  | 0.39        |maxDepth=5 | 0.5
| Initial #3        | Yes          | No           | 0.29         | 0.64                             | 0.68      |maxDepth=5| 0.5
| Grid Search #1        | Yes          | No           | 0.22         | 0.83                         | 0.35      |maxDepth = 2, maxBins = 20, minIter = 10| 0.5
| Final Candidate #1        | Yes          | No           | 0.19        | 0.98                      | 0.32      |maxDepth = 2, maxBins = 20, minIter = 10| 0.4


#### Random Forest
__Evaluation results in each tuning step__:

| Model              | undersampling | class weight | precision |&nbsp;&nbsp; recall &nbsp;&nbsp;   | f1-score |  params  | Threshold 
| ------------------ | --------------| ------------ | --------- | --------------------------------- | -------- | --------  | --------
| Initial #1         | No            | No           | 0         | 0                                 | 0        | numTrees=3 |0.5
| Initial #2         | No            | Yes          | 0.25      | 0.69                              | 0.37     | numTrees=20 |    0.5
| Initial #3         | Yes           | No           | 0.29      | 0.5                               | 0.37     | numTrees=20  | 0.5    
| Grid Search #1     | No           | Yes           |    0.23       |        0.65                   |   0.34   | maxDepth = 3, minWeightFractionPerNode=0.2, numTrees=20         |0.5
| Grid Search #2     | Yes           | No           |    0.22       |        0.81                   |   0.35   | maxDepth = 3, minWeightFractionPerNode=0.2, numTrees=20           | 0.5


#### Best Model Seletion 
__Chosen by best from each model__:


| Model              | undersampling | class weight | precision |&nbsp;&nbsp; recall &nbsp;&nbsp;   | f1-score | Params    | Threshold 
| ------------------ | --------------| ------------ | --------- | --------------------------------- | -------- |--------   | --------
| DecisionTree       | Yes | No |  0.25     | 0.66   | 0.36 | maxDepth = 5, minWeightFractionPerNode=0.0,minInstancesPerNode=1  | 0.5
| XGBoost            | Yes | No |   0.27     | 0.72                              |   0.39            |maxDepth=5 | 0.4
| RandomForest       | Yes | No | 0.22       |        0.81                   |   0.35      |maxDepth=5| 0.5
| GradientBoost      | Yes | No |0.19        | 0.98                      | 0.32       |maxDepth = 2, maxBins = 20, minIter = 10| 0.5


#### Model Comparison 

We  compared performance of 4 models, and decided to choose XGboost as our champion model considering the balance between precision and recall. 
Even fromb elow plots, we were able to see that XGboost is more stable than the other models looking at better ROC curve and more stable Precision & Recall curve.

<img src='https://docs.google.com/uc?export=download&id=1q9Jb_79f6RodK2doGkrVf10-Z3c8Wtnf' width=50%>

We developed and tested with models and finalized it into pipeline. Below is the link

#### [Please use this link to find the notebook.](https://adb-731998097721284.4.azuredatabricks.net/?o=731998097721284#notebook/1731553363141160/command/1858507102407110)

## 6 - Algorithm implementation - Logistic Regression from Scratch

#### [Please use this link to find the notebook that implements logistic regression model from Scratch.](https://adb-731998097721284.4.azuredatabricks.net/?o=731998097721284#notebook/1731553363137916/command/1731553363137929)

## 7 - Conclusions

#### Machine Learning at Scale

#### Learning from Machine Learning at Scale

*Test enough with samples before hitting the button!*
>This was first time for our group to make a model with a "big" data set. From EDA to data engineering to final modeling, we learned that we have to plan strategically about how to use our limited resources. For example, many visualization packages like matplotlib, plotly, and some seaborn plots do not work at scale. In cases when we wanted boxplots or histograms, we decided to randomly sample from the larger dataset. In cases where we could use visualizations at scale, we first vectorized the features.
Even when we wanted to create the new variables, it was hard to examine their effectiveness until we ran the whole model again. Therefore, it was important to test enough with samples before we hit the run button. In the modeling stage, Gridsearch and Crossvalidation part takes a lot of time thus, it was important to set the right parameter from the first. 
the running time varied by model. Decision Trees, especially, took a lot time relative to Random Forest. 

*Important to checkpoint, but remember storing also costs!*

>Even with our clusters at max workers and power, team members still had to communicate when simultaneously running models as there was noticable degradation in speed when everyone was modeling in parallel. Checkpointing, writing processed data/model outputs to cache and to storage parquets was crucial to reduce duplication of effort and long processing times. However, if we save without consideration, blob storage will exceed the budget, therefore, we had to carefully monitor storage limits.



#### Learning by Gap Analysis   
> After we applied  multiple measures as described above, we found that applying ensamble models led to the best result. In other words, better recall performance. However, with the test set, precision was lower than in the training result. This means that the ensemble model overfitted. Therefore, we learned that there is a trade off to applying advanced modeling in term of variance. Our next plan would be reduce this overfitting issue with more appropriate data sets and tuning techniques.


- Initial Model 
<img src='https://docs.google.com/uc?export=download&id=13K2-Uk9PbV1amDtNh4SuUJ_dQ5bhjzUX' width=40%>


- Final Model with XG Boost
>
<img src='https://docs.google.com/uc?export=download&id=1oOa8-HSV4P7VQHFbJka1h-59zJTWs_sO' width=40%>

#### Business Application

The top 5 airline carriers recommended for exclusivity agreements with EA are the following:

#### 1. AA - American Airlines
#### 2. UA - United Airlines
#### 3. WN - Southwest Airlines
#### 4. OO - Skywest Airlines
#### 5. DL - Delta Airlines

The team arrived at these carriers by first disqualifying the 8 worst airlines candidates with cancellation rates above 2%. 

|OP_CARRIER|numTotalFlights|numCancelled|pctCancelled|
|----------|---------------|------------|------------|
|        MQ|         917640|       37110|        4.04|
|        OH|         567761|       18696|        3.29|
|        EV|        1740081|       48842|        2.81|
|        YV|         443026|       12239|        2.76|
|        YX|         645239|       17792|        2.76|
|        9E|         503049|       10612|        2.11|
|        US|         198715|        4067|        2.05|
|        NK|         793419|       15101|         1.9|
|        B6|        1450596|       26544|        1.83|


T18 does not recommend that EA partner with the following due to their __cancellation__ record:

- MQ - American Eagle
- OH - Blue Streak
- EV - ExpressJet
- YV - Air Shuttle
- YX - Republic airways
- 9E - Endeavor air
- US - US airways

While many of these airlines offer low pricing, the cost of missing opportunities far outweigh any airfare savings. 

The second partnership qualifier was __delay probability__, which is an output from the best performing prediction model(XG Boost). 

Finally, __predictability__ of the carrier was weighed for rank order. Predictability is how close the predicted number of delayed flights align with the actual number of delayed flights. The reason that Delta was ranked lower even though it has one of the lower delay probabilities is because it is the least predictable carrier in the group.

From exploratory data analysis, the team found that seasonality has more predictive power than weather measurements. This was clear in the separation of box plots between the delayed and non-delayed classes and also in boxplot trends of delays aggregated by month and again by day of the week.

## 8 - Course concepts

#### Map Reduce for Gradient Descrent update
In order to implement Logistic Regression from scratch, we have to implement Gradient Descent steps to update the current model parameters. We applied Map Reduce method to implement Gradient update at scale. For every step, we first transform the DataRDD to the predictRDD via the sigmoid mapper function. Then, we compute the gradient using another mapper function to compute the dot product of all features and the delta between predicted value and actual value. Finally applying the regularzation parameters and the learning rate to form the new model.

#### Sparse and Dense representation for one-hot encoding 
In order to allow for further expansion of one-hot categories in future for modeling exploration and improvement, we make an optimization to expand to the full one-hot vector only when needed for computation. This will enable us for memory usage optimization, we process the RDD in its original form to store the one-hot indices in dense formation. Then it is broadcasted to all executers. During the subsequent dot product computations, we would transform the densed one-hot indices into the full one-hot vector dimensions for calculation. As a result, we never incur the memory overhead of storing the entire expanded one-hot encoded RDD in memory.

The picture below depicts our data tranformation strategy:

<div style="text-align: center; line-height: 10; padding-top: 30px;  padding-bottom: 30px;">
  <img src="https://i.postimg.cc/V6WD0rGK/one-hot-encode.png" alt='One Hot Encode' style="width: 500px" >
</div>

### Feature Selection / Assumptions
At the early phase of exploration, we put many efforts to select, impute and engineer weather conditions for modeling, yet could not acheive decent value of selected metric that aligned with our business case. Even with the help of feature importance out of Random Forest, it only allowed us graduated from the high dimensional raw features without giving high performance. We realized we shall engineer novel features using assumptions based on real life experience. For example, we assumed the delay status and operation efficiency of an airport could affect flights schedule. Therefore, three features were derived and used as airport operation metrics. Also, under the assumption that an aircraft delayed in its previous landing could delay its following flight, we used motif finding algorithm to track down an aircraft traces and restructured 30% of data points into features of the rest of the records. Combining assumptions based on real life experience with novel graph algorithms, we were able to acheive high recall score.

### Bias and variance Tradeoff 

Prediction errors are defined as the collection of bias, variance, and irreducible errors. High bias refers to when a model shows high inclination towards an outcome of a problem it seeks to solve. Variance error refers the sensitivity of models to slight fluctuations in the training data describes the variance. We have learned from the class Bias and variance are trade off. In other words, Increasing the bias leads to a decrease in variance. Suppose we reduce bias, and variance increases. We examined this reuslt from our ensamble model that it reduced the bias but increased the variance after we saw the test model result gotten worse.


reference: https://www.section.io/engineering-education/ensemble-bias-var/

## 9 - Novel Aspects

Thinking about real life scenarios, one major reason flight delays is that the previous flight operated by the same aircraft was delayed, resulting in the subsequent flights not being operated as scheduled. As seen in the EDA notebook, PageRank finds airport hubs. With more flights flying in and out, the scheduling of airport becomes crucial when it comes to flight delays. Derived from this finding, we are interested in finding out whether there are flight patterns being affected by airport hubs. To find such features in our current dataset, we utilized the motif finding function in GraphFrame library to find the potential consecutive flights operated by the same aircraft. With additional features of the source airport, such as weather data at two time points and airport evaluation scores, we built GBM model (GBM is the best performing model in our model enhancement process) to see if we could improve recall of predictions.

To find motifs in the master dataset, we use airports as vertices, arriving and departure flights as edges, weather and airport features as properties. The pattern of flights we are trying to find can be represented in a format as in '(a)-[ab]->(b); (b)-[bc]->(c)'. In this representation, 'a' is the source airport of previous flight, b is the destination airport of previous flight and source airport of the current flight that we are trying to predict delays on, and c is the destination airport of the current flight. Additional matching criteria were used ensure finding traces of the aircrafts within a propro time frame. 

In this exploration attempt, we assumed that not only the weather conditions in the source airport affects departure delay, but how airports schedule flights and operate also have effects, even though such features were not directly given. Three unique features were engineered by using the 2015-2019 data, including count, count_normal, and PageRank score. All three are being added to the model exploration. 

#### [Please use this link to find the notebook exploring this assumption using GBM model.](https://adb-731998097721284.4.azuredatabricks.net/?o=731998097721284#notebook/1858507102378535/command/1858507102399281)

## 10 - Limitations, Challenges, and Future Work

#### Limitations:

As we were wrapping up the modeling efforts, we realized that what we should have built for our client was a recommender system and what we had actually built was a prediction system. It was too late to pivot, so the recommendations that we arrived at amounted to advanced historical analytics that could have likely been derived with less sophisticated methodologies.  
 
With only weather and airline data, our findings of delay relationships were highly correlation based and we did not parse the data for natural experiments that might guide us towards casual relationships and stronger reasoning for our recommendations.

Airlines like Hawaiian exhibited very low delay probabilities and our model overpredicted the number of delays by a large margin. Our current hypothesis is that the number of Hawaiian flight operations is less than 10% of the larger carriers across comparable timeframes.
 
#### Challenges: 

Surprisingly, weather features in general and spefically ones such as wind speed, temperature, dew point did not hold much predictive power with respect to flight delays. Even though we have applied various feature engineering techniques, due to lack of domain knowledge in aviation, interpretation and usage of weather features might be not sufficiently ustilized to uncover their power with respect to predicting flight delays. This implies that other operational aspects of airlines and airports such as reliability, security, and density data sources might provide greater prediction power. However, such features were only derived from the airlines dataset in our modeling process. 

We were excited to try L1 regularization in logistic regression and PCA for dimension reduction but quickly found it difficult to map the one-hot encoded categorical features back to interpretable representations of the original features. 

The team actually reached out to a pilot and air traffic controller to help understand the nuances of flight delays but never received a reply.

It would have been ideal to gather feature importance from our best performing XGBoost models, though we were not able to extract the feature gains from pyspark's Mllib because the output is in Java, which does not translate well to Python.

#### Future work: 

Not only were the delay classes imbalanced, the activity per carrier was also weighted towards several popular airlines, so as we prepared to aggregate results by airline, we realized that the company that runs most flights will also have the most delays. In the future, we would scale/standardize this metric.

If we had the opportunity to improve on this project, we would certainly ask to interview subject matter experts. 


## Deepest thanks to Vini and the whole W261 team for opening our eyes, enlightening, and challenging us! You were great sherpas :)